In [ ]:
%%writefile ".env"

PROJECT_ID="feux_cq"

COMMUN_PATH = "C:/"
PATH_INFOCENTRE_APP = ${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/
PATH_ETUDE = ${COMMUN_PATH}Informatique/SIG/Donnees/Oeil/Traitement_Donnees/SURFACES_BRULEES_SENTINEL/2022/220502_PreparationFeu2020/
PROJECT_PATH =${COMMUN_PATH}Users/oriane.bruyere/Documents/Projet_feux/Script/Step1_detection_error/
DATA_CATALOG_DIR = ${PATH_INFOCENTRE_APP}projets/catalogFiles/ 
DATA_OUTPUT_DIR = ${PROJECT_PATH}output/
SIG_DATA_PATH = ${COMMUN_PATH}Informatique/SIG/Donnees/
DB_USER="jfnguyenvansoc"
DB_PWD="oeil"
DB_HOST="172.20.12.13"
DB_PORT=5432

DB_WORKSPACE="oeil_traitement"
DB_REF="oeil_reference"
DB_EXT="data_externe"
DB_SCHEMA = "feux_cq"
DB_SCHEMA_REF = "feux"

## Get missing burned areas files from INSIGHT server

In [ ]:
import logging
from dotenv import load_dotenv
from filecmp import dircmp
from os import listdir
import os
import filecmp 
import geopandas as gpd
import pandas as pd
import fiona
from sqlalchemy import create_engine
load_dotenv()
import subprocess
from tobler.util import h3fy
import h3
from shapely.geometry import MultiPolygon

## Open gpkg and add data to POSTGIS

In [ ]:
DATA_TEMP ='A:/FEUX_INSIGHT/2018/'

In [ ]:
fichiers_geopackages = [f for f in os.listdir(DATA_TEMP) if f.endswith('.gpkg')]
gdf_list = []

for fichier_geopackage in fichiers_geopackages:
    chemin_geopackage = os.path.join(DATA_TEMP, fichier_geopackage)
    couches = fiona.listlayers(chemin_geopackage)

    for couche in couches:
        gdf_couches = gpd.read_file(chemin_geopackage, layer=couche, driver='GPKG')
        
        if 'date_' in gdf_couches.columns:
            gdf_couches['date_']=pd.to_datetime(gdf_couches['date_'])
            gdf_couches['date_']=gdf_couches['date_'].dt.date
            gdf_couches['datetexte'] = gdf_couches['date_texte'].apply(lambda x: x[:-11] if len(x) > 8 else x)

        elif 'date_texte' or 'date' in gdf_couches.columns:
            gdf_couches.rename(columns={'date_texte': 'date_'}, inplace=True)
            gdf_couches.rename(columns={'date': 'date_'}, inplace=True)
            
            gdf_couches['datetexte'] = gdf_couches['date_'].apply(lambda x: x[:-11] if len(x) > 8 else x)
            gdf_couches['date_']=pd.to_datetime(gdf_couches['datetexte'])
            gdf_couches['date_']=gdf_couches['date_'].dt.date

    
        gdf_couches['nom'] = gdf_couches['nom'].apply(lambda x: x[:19] + x[30:] if len(x) > 30 else x)
        gdf_list.append(gdf_couches)

gdf = pd.concat(gdf_list, ignore_index=True, sort=False)
gdf=gdf.reset_index(drop=True)

print("Données fusionnées à partir des GeoPackages :")

### Convertir les surfaces detectees en type MULTIPOLYGONE

In [ ]:
# Vérifier si la géométrie est de type Polygon
if gdf['geometry'].geom_type[0] == 'Polygon':
    gdf['geometry'] = gdf['geometry'].apply(lambda poly: MultiPolygon([poly]))

    print(gdf)
else:
    print("La géométrie n'est pas de type Polygon.")

### Ajouter une identifiant H3 et créer un identifiant unique

In [ ]:
H3_res = 12
def geo_to_h3(row):
  return h3.geo_to_h3(row.lat,row.lon,resolution = H3_res)

In [ ]:
gdf=gdf.to_crs(4326) ## change coordinates to WGS84
gdf=gdf.set_crs("EPSG:4326")

gdf['centroides']=gdf['geometry'].centroid
gdf['lon'] = gdf.centroides.x
gdf['lat'] = gdf.centroides.y

gdf['h3_id'] = gdf.apply(geo_to_h3,axis=1)

gdf['surface_id_h3']=0
for i in range(len(gdf)):
  gdf['surface_id_h3'][i]=str(gdf['nom'][i][-10:])+"_"+str(gdf['datetexte'][i])+"_"+str(gdf['h3_id'][i])

## Création de la base de donnée dans POSTGIS

In [ ]:
gdf = gpd.GeoDataFrame(gdf.copy(), crs=gdf.crs, geometry=gdf['geometry'])
gdf=gdf.to_crs(3163) ## change coordinates to RGNC
gdf=gdf.set_crs("EPSG:3163")
gdf=gdf[["nom", "province", "commune", "surface", "geometry","date_","surface_id_h3"]]

In [ ]:
from sqlalchemy import text
table = "surfaces_brulees_brute_control"

conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')
gdf.to_postgis(table, conex,schema=os.getenv("DB_SCHEMA"),if_exists='append')